In [19]:
import numpy as np
import pandas as pd
import webhoseio
from aylienapiclient import textapi
import re
from urlparse import urlparse

In [2]:
# Ashwin's key - we only get 1000 free requests a month, so be cautious with the number of times you perform queries
webhoseio.config(token="456af32b-3c58-455d-b9f8-90875bfc8f58") 

In [16]:
client = textapi.Client("4657dfa6", "bbfd5faa23a1020d92e946dd83bce6b7")

In [22]:
output_ex = webhoseio.query("filterWebData", {"q":"thread.title:(Uber isn't sure if it can 'remain a viable business' without building self-driving cars) site:businessinsider.com"})

## Get the title and site_url of an article given a url

In [17]:
def getTitleAndSiteUrl(url):
    extract = client.Extract({"url": url, "best_image": False})
    title = extract['title']
    site_url = re.sub('www.', '', urlparse(url).hostname)
    
    return title, site_url

## Create the dataframe with the 20 features that are in the Kaggle Data Set. Also create the function that will add rows to the dataframe

In [14]:
# all the columns from the Kaggle data set
cols = ["uuid", "ord_in_thread", "author", "published", "title", "text", "language", "crawled", "site_url", "country", "domain_rank", "thread_title", "spam_score", "main_img_url", "replies_count", "participants_count", "likes", "comments", "shares", "type"]
df = pd.DataFrame(columns = cols)
df

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type


In [10]:
def addRow(df, site_url, title):
    output = webhoseio.query("filterWebData", {"q":"thread.title:(" + title + ") site:" + site_url})
    post = output["posts"][0]
    
    thread = post["thread"]
    social = thread["social"]["facebook"]
    
    uuid = post["uuid"]
    ord_in_thread = post["ord_in_thread"]
    author = post["author"]
    published = post["published"]
    title = post["title"]
    text = post["text"]
    language = post["language"]
    crawled = post["crawled"]
    site_url = thread["site"]
    country = thread["country"]
    domain_rank = thread["domain_rank"]
    thread_title = thread["title"]
    spam_score = thread["spam_score"]
    img = thread["main_image"]
    replies = thread["replies_count"]
    participants = thread["participants_count"]
    likes = social["likes"]
    comments = social["comments"]
    shares = social["shares"]
    
    #data = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0,0,0,0,0,0,0,0]]
    data = [[uuid, ord_in_thread, author, published, title, text, language, crawled, site_url, country, domain_rank, thread_title, spam_score, img, replies, participants, likes, comments, shares, "N/A"]]
    df_row = pd.DataFrame(data, columns=cols)
    #print(df_row)
    df = df.append(df_row, ignore_index=True)
    return df

In [21]:
title, site_url = getTitleAndSiteUrl("http://www.businessinsider.com/uber-questions-future-of-being-a-viable-business-without-self-driving-cars-2017-4")
print(title)
print(site_url)

df = addRow(df, site_url, title)
df

Uber isn't sure if it can 'remain a viable business' without building self-driving cars
businessinsider.com


,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,b046f9bc0e5160022bf7b5b1836e2aa9b3671db5,0.0,Biz Carson,2017-04-08T15:11:24.611+03:00,Uber isn't sure if it can 'remain a viable bus...,print \nUber CEO Travis Kalanick Reuters Staff...,english,2017-04-08T15:11:24.611+03:00,businessinsider.com,US,283.0,Uber isn't sure if it can 'remain a viable bus...,0.0,http://static1.businessinsider.com/image/58e82...,0.0,1.0,0.0,0.0,0.0,N/A
